In [ ]:
import csv
import requests
from typing import List, Dict


def generate_description(part_info: str) -> str:
    payload = {
        "inputs": f"Generate a concise description for an auto part: {part_info}",
        "parameters": {"max_length": 100, "min_length": 30}
    }
    response = requests.post(API_KEY, headers=headers, json=payload)
    return response.json()[0]['generated_text']

def process_parts(parts: List[Dict]) -> List[Dict]:
    processed_parts = []
    for part in parts:
        description = generate_description(f"{part['name']} for {part['year']} {part['make']} {part['model']}")
        processed_part = {
            'part_number': part['part_number'],
            'name': part['name'],
            'description': description,
            'price': part['price'],
            'quantity': part['quantity'],
            'year': part['year'],
            'make': part['make'],
            'model': part['model']
        }
        processed_parts.append(processed_part)
    return processed_parts

def save_to_csv(parts: List[Dict], filename: str):
    keys = parts[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(parts)

# Example usage
sample_parts = [
    {
        'part_number': 'BP-2024',
        'name': 'Brake Pads',
        'price': 45.99,
        'quantity': 100,
        'year': 2022,
        'make': 'Toyota',
        'model': 'Camry'
    },
    {
        'part_number': 'OL-1010',
        'name': 'Oil Filter',
        'price': 12.99,
        'quantity': 200,
        'year': 2023,
        'make': 'Honda',
        'model': 'Civic'
    }
]

processed_parts = process_parts(sample_parts)
save_to_csv(processed_parts, 'processed_parts.csv')

In [ ]:
from llama_index.tools import FunctionTool

def process_and_save_parts(parts: List[Dict], output_filename: str) -> str:
    processed_parts = process_parts(parts)
    save_to_csv(processed_parts, output_filename)
    return f"Processed {len(processed_parts)} parts and saved to {output_filename}"

parts_processor_tool = FunctionTool.from_defaults(
    fn=process_and_save_parts,
    name="process_and_save_parts",
    description="Processes a list of auto parts, generates descriptions, and saves to a CSV file"
)

In [ ]:
agent = ReActAgent.from_tools(
    [index.as_query_engine(), parts_processor_tool],
    llm=your_llm,
    verbose=True
)